# AI & Security Project

**evaluating_pretrained_models.ipynb**: in this notebook we evaluate several pre-trained image classification models on our dataset 'TinyImageNet'. This evaluation happens before we implement FGSM to cause misclassification


In [1]:
import os
import json
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torch.utils import data
from tqdm.notebook import tqdm

## Step 0: Configurations


In [2]:
# Define constants
DATASET_PATH = r"./data/TinyImageNet-sad/"  # Adjust this to the correct path where the dataset is stored
CHECKPOINT_PATH = r"./models/"
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transformations
plain_transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)]
)

# Construct the path to the dataset
imagenet_path = os.path.join(DATASET_PATH, "TinyImageNet")
assert os.path.isdir(imagenet_path), (
    f'Could not find the ImageNet dataset at the expected path: "{imagenet_path}". '
    "Please make sure the dataset is downloaded and the path is correct."
)

## Step 1: dataset and libraries


In [3]:
# Load the dataset
dataset = torchvision.datasets.ImageFolder(
    root=imagenet_path, transform=plain_transforms
)
data_loader = data.DataLoader(
    dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=8
)

In [4]:
# Load label names
label_list_path = os.path.join(imagenet_path, "label_list.json")
assert os.path.isfile(
    label_list_path
), f'Label list file not found at "{label_list_path}".'

with open(label_list_path, "r") as f:
    label_names = json.load(f)

## Step 2: model functions

In [5]:
def load_model(model_func, trainable=False):
    """Load a pretrained model given its name."""
    model = model_func()
    model = model.to(device)
    model.eval()

    # Control gradient computation
    for p in model.parameters():
        p.requires_grad = trainable

    return model


def eval_model(dataset_loader, model, img_func=None):
    """Evaluate the model on the given dataset loader."""
    tp, tp_5, counter = 0.0, 0.0, 0.0
    for imgs, labels in tqdm(dataset_loader, desc="Validating...", leave=False):
        imgs = imgs.to(device)
        labels = labels.to(device)
        if img_func is not None:
            imgs = img_func(imgs, labels)
        with torch.no_grad():
            preds = model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[..., None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item() / counter
    top5 = tp_5.float().item() / counter
    print(f"\tTop-1 error: {(100.0 * (1 - acc)):4.2f}%")
    print(f"\tTop-5 error: {(100.0 * (1 - top5)):4.2f}%")
    return acc, top5

In [6]:
from torchvision import models
from torchvision.models import ResNet18_Weights, ResNet50_Weights, ResNet152_Weights, VGG16_Weights, VGG19_Weights, RegNet_Y_128GF_Weights, ViT_H_14_Weights, ViT_L_16_Weights


list_of_models = {
    # "RegNet_Y_128GF": lambda: models.regnet_y_128gf(weights=RegNet_Y_128GF_Weights.DEFAULT),		# 2.40GB
    # "ViT_H_14": lambda: models.vit_h_14(weights=ViT_H_14_Weights.DEFAULT),						# 2.36GB
    # "ViT_L_16": lambda: models.vit_l_16(weights=ViT_L_16_Weights.DEFAULT),						# 1.13GB
    "ResNet18": lambda: models.resnet18(weights=ResNet18_Weights.DEFAULT),
    "ResNet50": lambda: models.resnet50(weights=ResNet50_Weights.DEFAULT),
    "ResNet152": lambda: models.resnet152(weights=ResNet152_Weights.DEFAULT),
    "VGG16": lambda: models.vgg16(weights=VGG16_Weights.DEFAULT),
    "VGG19": lambda: models.vgg19(weights=VGG19_Weights.DEFAULT),
}


for model_name, model_func in list_of_models.items():
    print(f"\nEvaluating {model_name}:")
    model = load_model(model_func)
    _ = eval_model(data_loader, model)


Evaluating ResNet18:


Validating...:   0%|          | 0/157 [00:00<?, ?it/s]

	Top-1 error: 24.00%
	Top-5 error: 6.76%

Evaluating ResNet50:


Validating...:   0%|          | 0/157 [00:00<?, ?it/s]

	Top-1 error: 13.24%
	Top-5 error: 1.96%

Evaluating ResNet152:


...

	Top-1 error: 8.34%
	Top-5 error: 0.64%

Evaluating VGG16:


Validating...:   0%|          | 0/157 [00:00<?, ?it/s]

	Top-1 error: 21.92%
	Top-5 error: 5.62%

Evaluating VGG19:


Validating...:   0%|          | 0/157 [00:00<?, ?it/s]

	Top-1 error: 21.18%
	Top-5 error: 5.14%
